Interpolacja Hermite'a

Zakladam, ze input jest postaci

xs = [x1, x2, x3, x4, ...]
ys = [ [y1, y1', y1'', ...], ... ], czyli do kazdego x-a przypisana jest lista y-ów, z której każdy kolejny oznacza wartość w kolejnej pochodnej

In [54]:
import math

In [27]:
def pretty_print(array):
    
    for row in array:
        for el in row:
            print(el, end = ' ')
        print("")

In [110]:
def hermite_interpolation(xs, ys, x):

    x_map = []
    for i, mi in enumerate(ys):
        for _ in range(len(mi)):
            x_map.append(xs[i])

    #Wyznaczam sumę krotności
    m = sum([len(mi) for mi in ys])
    
    #Tworzę początkową macierz i wypełniam znanymi wartościami
    A = [[ None for j in range(m)] for i in range(m)]
    
    #Wypełniam macierz znanym wartościami f(x)
    current_row = 0
    for mi in ys:
        for j in range(len(mi)):
            A[current_row][0] = mi[0]
            current_row += 1
            
    #Wypełniam macierz znanymi wartościami pochodnych
    
    row = 0
    for mi in ys:
        for i in range(len(mi)):
            for j in range(1, i + 1):
                A[row][j] = mi[j] / math.factorial(j)
            row += 1
            
    #Wypełniam macierz wylioczonymi różnicami i wyliczam wspolczynniki
    coefficients = [A[0][0]]
    for i in range(1, m): #Aktualna pozycja do ktorej zmierzamy
        for j in range(1, i + 1):
            if A[i][j] == None:
                A[i][j] = (A[i][j - 1] - A[i - 1][j - 1]) / (x_map[i] - x_map[i - j])
        coefficients.append(A[i][i])
        
        
    pretty_print(A)
    
    diffs = [x - xi for xi in x_map]
    
    w = coefficients[0]
    
    for i in range(1, m):
        
        temp = coefficients[i]
        
        for j in range(0, i):
            temp *= diffs[j]
            
        w += temp
        
    return w  

In [113]:
#Testowe wartosci
xs = [1, 2]
ys = [[1, 4], [3, 1, 1]]

moja = hermite_interpolation(xs, ys, 123)

print("mojsza: ", moja)


1 None None None None 
1 4.0 None None None 
3 2.0 -2.0 None None 
3 1.0 -1.0 1.0 None 
3 1.0 0.5 1.5 0.5 
1.0 None None None None 
1.0 4.0 None None None 
3.0 2.0 -2.0 None None 
3.0 1.0 -1.0 1.0 None 
3.0 1.0 0.5 1.5 0.5 
mojsza:  110730007.0
twojsza:  110730007.0
